# Trainer

## Prepare Dataset

In [1]:
from torchhk.datasets import *

In [2]:
mnist = Datasets("MNIST", root='./data',
                 transform_train=transforms.ToTensor(), 
                 transform_test=transforms.ToTensor())


train_loader, _ = mnist.get_loader(batch_size=128)

Data Loaded!
Train Data Length : 60000
Test Data Length : 10000


## Prepare Model

In [3]:
import torch
import torch.nn as nn

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 16, 5),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )       
        
    def forward(self,x):
        out = self.conv_layer(x)
        out = out.view(-1,64*3*3)
        out = self.fc_layer(out)

        return out

In [5]:
from torchhk.trainer import BaseTrainer

In [6]:
model = CNN().cuda()
trainer = BaseTrainer(model)

In [7]:
trainer.train(train_loader=train_loader, max_epoch=4,
              optimizer="SGD(lr=0.01, momentum=0.9)",
              scheduler="MultiStepLR(milestones=[4, 6], gamma=0.1)", scheduler_type="Epoch",
              # or scheduler="Step([2, 4], 0.1)"
              save_type="Epoch", save_path="./_models/", save_overwrite=False,
              record_type="Epoch")

[BaseTrainer]
Training Information.
-Epochs: 4
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.01
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
-Scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x000001EFEA0DA6A0>
-Save Path: ./_models/
-Save Type: Epoch
-Record Type: Epoch
-Device: cuda:0
------------------------------------              
Epoch   Loss     Acc       lr       
1       0.5816   80.4270   0.0100   
------------------------------------
2       0.0829   97.4058   0.0100                 
------------------------------------
3       0.0541   98.2806   0.0100                 
------------------------------------
4       0.0426   98.6612   0.0100                 
------------------------------------
Total Epoch: 4
Time Elapsed: 0:00:30.279924
Min(epoch)/Max(epoch): 
-Loss: 0.0426(4)/0.5816(1)
-Acc: 80.4270(1)/98.6612(4)
-lr: 0.0100(1)/0.0100(1)
------------------------------------


In [8]:
trainer.train(train_loader=train_loader, max_epoch=8, start_epoch=4,
              optimizer="SGD(lr=0.01, momentum=0.9)",
              scheduler="MultiStepLR(milestones=[4, 6], gamma=0.1)", scheduler_type="Epoch",
              # or scheduler="Step([2, 4], 0.1)"
              save_type="Epoch", save_path="_models/", save_overwrite=True,
              record_type="Epoch")

[BaseTrainer]
Training Information.
-Epochs: 8
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.01
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
-Scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x000001EFEA0B00F0>
-Save Path: _models/
-Save Type: Epoch
-Record Type: Epoch
-Device: cuda:0


C:\Users\slcf\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:122: UserWarning:

Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate



------------------------------------              
Epoch   Loss     Acc       lr       
5       0.0253   99.2772   0.0010   
------------------------------------
6       0.0229   99.3640   0.0010                 
------------------------------------
7       0.0213   99.4040   0.0001                 
------------------------------------
8       0.0209   99.4274   0.0001                 
------------------------------------
Total Epoch: 8
Time Elapsed: 0:00:29.144723
Min(epoch)/Max(epoch): 
-Loss: 0.0209(8)/0.0253(5)
-Acc: 99.2772(5)/99.4274(8)
-lr: 0.0001(7)/0.0010(5)
------------------------------------


In [9]:
trainer.save_all("final")

Saving Model
...Saved as pth to final.pth !
Saving Records
...Saved as csv to final.csv !
